In [25]:
import pandas as pd
import numpy as np

In [26]:
data = pd.read_csv("lsh_data.csv")
data.head(3)

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...


In [27]:
data.describe()

,category,text
count,2215,2225
unique,5,2126
top,sport,ferguson rues failure to cut gap boss sir alex...
freq,509,2


In [28]:
data['category'].value_counts()

sport            509
business         508
politics         415
tech             399
entertainment    384
Name: category, dtype: int64

- The data has collection of news across various categories
- Supervised learning
- X -> text
- Y -> category

In [29]:
data[pd.isnull(data).any(axis=1)]

,category,text
2215,NaN,junk e-mails on relentless rise spam traffic i...
2216,NaN,top stars join us tsunami tv show brad pitt r...
2217,NaN,rings of steel combat net attacks gambling is ...
2218,NaN,davies favours gloucester future wales hooker ...
2219,NaN,beijingers fume over parking fees choking traf...
2220,NaN,cars pull down us retail figures us retail sal...
2221,NaN,kilroy unveils immigration policy ex-chatshow ...
2222,NaN,rem announce new glasgow concert us band rem h...
2223,NaN,how political squabbles snowball it s become c...
2224,NaN,souness delight at euro progress boss graeme s...


 - The last 10 data points are for testing

In [30]:
test_data = data[pd.isnull(data).any(axis=1)]

In [31]:
train_data = data.dropna()

In [34]:
print("Train Data")
print("Number of data points"+str(len(train_data)))
print(train_data.head(2))
print("\n")
print("Test Data")
print("Number of data points"+str(len(test_data)))
print(test_data.head(2))

Train Data
Number of data points2215
   category                                               text
0      tech  tv future in the hands of viewers with home th...
1  business  worldcom boss  left books alone  former worldc...


Test Data
Number of data points10
     category                                               text
2215      NaN  junk e-mails on relentless rise spam traffic i...
2216      NaN  top stars join us tsunami tv show brad pitt  r...
